In [459]:
import pandas as pd
import numpy as np
import os
import regex as re

In [460]:
weather_file = 'weather_data_2019_2021.csv'

In [461]:
weather = pd.read_csv(weather_file, parse_dates=['Date'])
weather.head()

,Date,Time,Temperature (°C),Weather,Wind,Wind Direction,Humidity,Barometer (mbar),Visibility (km)
0,2019-02-01,12:00 SAT6 01 Tháng hai p.,26,Passing clouds.,9 km/h,Wind blowing from 130° Southeast to Northwest,70%,1013 mbar,NaN
1,2019-02-01,12:30 SA,26,Passing clouds.,9 km/h,Wind blowing from 130° Southeast to Northwest,74%,1013 mbar,NaN
2,2019-02-01,1:00 SA,26,Passing clouds.,11 km/h,Wind blowing from 130° Southeast to Northwest,74%,1013 mbar,NaN
3,2019-02-01,1:30 SA,25,Passing clouds.,9 km/h,Wind blowing from 140° Southeast to Northwest,79%,1012 mbar,NaN
4,2019-02-01,2:00 SA,25,Passing clouds.,11 km/h,Wind blowing from 130° Southeast to Northwest,79%,1012 mbar,NaN


In [462]:
def read_file(weather_file):
    return pd.read_csv(weather_file, parse_dates=['Date'])

In [463]:
# nối các file lại với nhau

weather = read_file('weather_data_2019_2021.csv') 
weather = pd.concat( [weather, read_file('weather_data_2021_2023.csv')]) 
weather = pd.concat( [weather, read_file('weather_data_2023_2025.csv')]) 
weather = pd.concat( [weather, read_file('weather_data_2025_2025.csv')]) 

len(weather)

101921

In [464]:
# đổi tên các cột
weather.columns = ['date', 'time', "temp", "weather", "wind", "wind_direction", "humidity", "pressure", "visibility"]
weather.head()

,date,time,temp,weather,wind,wind_direction,humidity,pressure,visibility
0,2019-02-01,12:00 SAT6 01 Tháng hai p.,26,Passing clouds.,9 km/h,Wind blowing from 130° Southeast to Northwest,70%,1013 mbar,NaN
1,2019-02-01,12:30 SA,26,Passing clouds.,9 km/h,Wind blowing from 130° Southeast to Northwest,74%,1013 mbar,NaN
2,2019-02-01,1:00 SA,26,Passing clouds.,11 km/h,Wind blowing from 130° Southeast to Northwest,74%,1013 mbar,NaN
3,2019-02-01,1:30 SA,25,Passing clouds.,9 km/h,Wind blowing from 140° Southeast to Northwest,79%,1012 mbar,NaN
4,2019-02-01,2:00 SA,25,Passing clouds.,11 km/h,Wind blowing from 130° Southeast to Northwest,79%,1012 mbar,NaN


In [465]:
# Convert the 'Date' column to datetime format
weather['date'] = pd.to_datetime(weather['date'], format='%Y-%m-%d')


In [466]:
# Chuyển đổi giờ từ 12:30 SA thành 00:30 cho tới 11:30 CH thành 23:30
def convert_time_format(time_str):
    if 'SA' in time_str:
        return pd.to_datetime(time_str.split('SA')[0] + ' AM', format='%I:%M %p').time().strftime("%H:%M")
    elif 'CH' in time_str:
        return pd.to_datetime(time_str.split('CH')[0] + ' PM', format='%I:%M %p').time().strftime("%H:%M")
    
weather['time'] = weather['time'].apply(convert_time_format)
weather['datetime'] = pd.to_datetime(weather['date'].astype(str) + ' ' + weather['time'], format='%Y-%m-%d %H:%M')

In [467]:
def convert_wind(wind):
    if isinstance(wind, str):
        if "No wind" in wind:
            return 0
        else:
            return wind.split()[0]
    return None  


weather['wind'] = weather['wind'].apply(convert_wind)

In [468]:
def extract_degree(wind):
    if isinstance(wind, str):
        match = re.search(r'from (\d+)°', wind)
        if match:
            return int(match.group(1))
    return None
weather['wind_direction'] = weather['wind_direction'].apply(extract_degree)

In [469]:
weather['humidity'] = weather['humidity'].str.replace('%', '').astype(float)
weather['pressure'] = weather['pressure'].str.replace('mbar', '').astype(float)

In [470]:
print(len(weather))
len(weather[weather['visibility'].isna()])


101921


72738

In [471]:
weather = weather.drop(columns=['visibility'])

In [472]:
weather['weather'].value_counts()

weather
Passing clouds.                             50512
Scattered clouds.                           19011
Partly sunny.                               11910
Clear.                                       3660
Partly cloudy.                               3263
                                            ...  
Rain showers. Low clouds.                       1
Scattered showers. More clouds than sun.        1
Scattered showers. Overcast.                    1
Sprinkles. Clear.                               1
Thunderstorms. Low clouds.                      1
Name: count, Length: 108, dtype: int64

In [473]:
# weather[weather['wind'] == None]
weather.tail()

,date,time,temp,weather,wind,wind_direction,humidity,pressure,datetime
3629,2025-04-20,04:30,28,Passing clouds.,7,160.0,84.0,1008.0,2025-04-20 04:30:00
3630,2025-04-20,05:00,28,Passing clouds.,9,160.0,84.0,1009.0,2025-04-20 05:00:00
3631,2025-04-20,05:30,28,Passing clouds.,7,140.0,84.0,1009.0,2025-04-20 05:30:00
3632,2025-04-20,06:00,28,Passing clouds.,9,150.0,84.0,1009.0,2025-04-20 06:00:00
3633,2025-04-20,06:30,28,Passing clouds.,9,140.0,84.0,1009.0,2025-04-20 06:30:00


In [474]:
full_weather = pd.DataFrame({'datetime': pd.date_range(start='2021-01-01',
                                end='2025-03-30',
                                freq='30min')
})

full_weather = full_weather.merge(weather, on='datetime', how='left')
# full_weather[full_weather['date'] == '2021-01-05']

full_weather[full_weather['datetime'] == '2021-01-05 04:00:00']

,datetime,date,time,temp,weather,wind,wind_direction,humidity,pressure
200,2021-01-05 04:00:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [475]:
# full_weather[full_weather['date'] == '2021-01-05']

In [476]:
full_weather[full_weather['weather'].isna()]

,datetime,date,time,temp,weather,wind,wind_direction,humidity,pressure
23,2021-01-01 11:30:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,2021-01-05 04:00:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
230,2021-01-05 19:00:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
232,2021-01-05 20:00:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358,2021-01-08 11:00:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
73941,2025-03-21 10:30:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73942,2025-03-21 11:00:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73943,2025-03-21 11:30:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74083,2025-03-24 09:30:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [477]:
import numpy as np

# Lấy các chỉ số bị NaN trong cột 'weather'
nan_indices = full_weather[full_weather['weather'].isna()].index.to_numpy()

# Tính hiệu giữa các chỉ số kế tiếp
diffs = np.diff(nan_indices)

# Nếu hiệu == 1, thì đang ở trong một chuỗi liên tiếp
groups = []
start = nan_indices[0]

for i in range(1, len(nan_indices)):
    if diffs[i-1] != 1:
        end = nan_indices[i-1]
        groups.append((start, end))
        start = nan_indices[i]
groups.append((start, nan_indices[-1]))  # Đừng quên nhóm cuối cùng

# In ra các đoạn NaN liên tiếp
for i, (start, end) in enumerate(groups, 1):
    print(f"Chuỗi {i}: từ index {start} đến {end} ({end - start + 1} dòng)")


Chuỗi 1: từ index 23 đến 23 (1 dòng)
Chuỗi 2: từ index 200 đến 200 (1 dòng)
Chuỗi 3: từ index 230 đến 230 (1 dòng)
Chuỗi 4: từ index 232 đến 232 (1 dòng)
Chuỗi 5: từ index 358 đến 358 (1 dòng)
Chuỗi 6: từ index 376 đến 376 (1 dòng)
Chuỗi 7: từ index 392 đến 392 (1 dòng)
Chuỗi 8: từ index 440 đến 440 (1 dòng)
Chuỗi 9: từ index 494 đến 494 (1 dòng)
Chuỗi 10: từ index 536 đến 536 (1 dòng)
Chuỗi 11: từ index 616 đến 616 (1 dòng)
Chuỗi 12: từ index 628 đến 628 (1 dòng)
Chuỗi 13: từ index 662 đến 662 (1 dòng)
Chuỗi 14: từ index 724 đến 725 (2 dòng)
Chuỗi 15: từ index 734 đến 734 (1 dòng)
Chuỗi 16: từ index 808 đến 808 (1 dòng)
Chuỗi 17: từ index 850 đến 850 (1 dòng)
Chuỗi 18: từ index 878 đến 878 (1 dòng)
Chuỗi 19: từ index 892 đến 892 (1 dòng)
Chuỗi 20: từ index 904 đến 904 (1 dòng)
Chuỗi 21: từ index 953 đến 953 (1 dòng)
Chuỗi 22: từ index 1000 đến 1000 (1 dòng)
Chuỗi 23: từ index 1058 đến 1058 (1 dòng)
Chuỗi 24: từ index 1120 đến 1120 (1 dòng)
Chuỗi 25: từ index 1166 đến 1166 (1 dòng)
Chu

In [478]:
# weather[weather['date'] == '2025-03-21']

In [479]:
full_weather['date'] = full_weather['datetime'].dt.date
full_weather['time'] = full_weather['datetime'].dt.strftime("%H:%M")

,datetime,date,time,temp,weather,wind,wind_direction,humidity,pressure


------------------------

In [481]:
aqi = pd.read_csv('openweather_aqi_hcm.csv', parse_dates=['datetime_utc'])

In [482]:
aqi.head()

,datetime_utc,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,2020-11-25 01:00:00,5,1922.61,51.86,22.96,1.61,53.88,66.82,88.41,14.69
1,2020-11-25 02:00:00,5,2243.04,62.59,34.27,4.87,73.43,72.41,96.33,18.75
2,2020-11-25 03:00:00,5,2296.45,59.90,56.21,13.41,94.41,78.21,104.83,23.31
3,2020-11-25 04:00:00,4,1188.28,22.13,55.52,65.09,82.02,47.59,62.52,13.05
4,2020-11-25 05:00:00,5,1001.36,10.51,52.09,123.02,82.02,56.25,70.02,11.40


In [483]:
# aqi['date'] = aqi['datetime_utc'].dt.date
# aqi['date'] = pd.to_datetime(aqi['date'], format='%Y-%m-%d')
# aqi['time'] = aqi['datetime_utc'].dt.strftime("%H:%M")
# aqi['time'] = pd.to_datetime(aqi['time'], format='%H:%M').dt.time

In [484]:
# đổi tên datetime_utc thành datetime
aqi['datetime'] = aqi['datetime_utc'].dt.strftime("%Y-%m-%d %H:%M")
aqi = aqi.drop(columns=['datetime_utc'])

In [485]:
aqi

,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3,datetime
0,5,1922.61,51.86,22.96,1.61,53.88,66.82,88.41,14.69,2020-11-25 01:00
1,5,2243.04,62.59,34.27,4.87,73.43,72.41,96.33,18.75,2020-11-25 02:00
2,5,2296.45,59.90,56.21,13.41,94.41,78.21,104.83,23.31,2020-11-25 03:00
3,4,1188.28,22.13,55.52,65.09,82.02,47.59,62.52,13.05,2020-11-25 04:00
4,5,1001.36,10.51,52.09,123.02,82.02,56.25,70.02,11.40,2020-11-25 05:00
...,...,...,...,...,...,...,...,...,...,...
37620,2,286.14,0.00,4.48,63.04,1.53,11.58,17.27,2.77,2025-04-09 13:00
37621,2,299.63,0.00,4.82,61.99,1.57,12.05,17.82,2.96,2025-04-09 14:00
37622,2,311.42,0.00,5.11,60.28,1.56,12.16,17.62,3.02,2025-04-09 15:00
37623,2,320.84,0.00,5.33,57.81,1.48,11.95,16.92,3.04,2025-04-09 16:00


-------------

In [486]:
full_weather['datetime'] = pd.to_datetime(full_weather['datetime'])
aqi['datetime'] = pd.to_datetime(aqi['datetime'])

In [487]:
merged_data = pd.merge(full_weather, aqi, on='datetime', how='right') 
merged_data.tail(10)

,datetime,date,time,temp,weather,wind,wind_direction,humidity,pressure,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
37615,2025-04-09 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,212.34,0.07,1.54,63.97,0.86,6.40,8.91,1.16
37616,2025-04-09 09:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,218.33,0.04,1.88,62.95,0.89,6.51,9.00,1.22
37617,2025-04-09 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,231.70,0.02,2.56,62.97,1.02,7.69,11.12,1.63
37618,2025-04-09 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,254.62,0.01,3.51,63.43,1.29,9.55,14.29,2.24
37619,2025-04-09 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,272.94,0.00,4.11,63.58,1.45,10.79,16.19,2.58
37620,2025-04-09 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,286.14,0.00,4.48,63.04,1.53,11.58,17.27,2.77
37621,2025-04-09 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,299.63,0.00,4.82,61.99,1.57,12.05,17.82,2.96
37622,2025-04-09 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,311.42,0.00,5.11,60.28,1.56,12.16,17.62,3.02
37623,2025-04-09 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,320.84,0.00,5.33,57.81,1.48,11.95,16.92,3.04
37624,2025-04-09 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,329.30,0.00,5.52,55.17,1.35,11.69,16.10,3.10


In [488]:
merged_data[merged_data['datetime'] == '2021-01-05 04:00:00']

,datetime,date,time,temp,weather,wind,wind_direction,humidity,pressure,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
987,2021-01-05 04:00:00,2021-01-05,04:00,NaN,NaN,NaN,NaN,NaN,NaN,4,1054.76,17.21,50.04,60.08,73.43,40.75,51.89,9.75


In [489]:
len(merged_data)

37625

In [490]:
# Lấy ngày từ 2021-01-01 trở đi
merge_data = merged_data.copy()
merge_data['date'] = pd.to_datetime(merge_data['date'])

merge_data = merge_data[merge_data['date'] >= '2021-01-01']
merge_data = merge_data[merge_data['date'] <= '2025-03-29']

In [491]:
merge_data.tail()

,datetime,date,time,temp,weather,wind,wind_direction,humidity,pressure,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
37506,2025-03-29 19:00:00,2025-03-29,19:00,29.0,Clear.,19,160.0,74.0,1007.0,2,407.22,0.03,22.28,26.82,17.88,11.50,16.64,3.17
37507,2025-03-29 20:00:00,2025-03-29,20:00,29.0,Clear.,17,150.0,74.0,1008.0,2,407.22,0.06,23.31,23.60,19.55,11.78,16.99,3.29
37508,2025-03-29 21:00:00,2025-03-29,21:00,28.0,Passing clouds.,19,130.0,79.0,1009.0,2,413.89,0.11,24.33,20.56,21.22,12.07,17.43,3.36
37509,2025-03-29 22:00:00,2025-03-29,22:00,28.0,Passing clouds.,19,130.0,79.0,1010.0,2,423.91,0.23,25.70,16.63,23.60,12.11,17.60,3.51
37510,2025-03-29 23:00:00,2025-03-29,23:00,28.0,Passing clouds.,17,130.0,79.0,1010.0,2,460.62,0.79,29.13,9.92,28.13,12.69,18.41,4.12


In [492]:
len(merge_data)

36624

In [493]:
# Đếm xem thử 1 ngày có đủ 24 giờ không?
def check_full_day(group):
    return len(group) != 24
# Group by date and check if each group has 24 records
full_days = merge_data.groupby(merge_data['date'].dt.date).filter(check_full_day)
full_days
                

,datetime,date,time,temp,weather,wind,wind_direction,humidity,pressure,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
1511,2021-01-27 00:00:00,2021-01-27,00:00,25.0,Passing clouds.,7,50.0,74.0,1009.0,5,4058.84,91.20,21.76,0.03,34.33,202.29,254.51,13.81
1512,2021-01-28 01:00:00,2021-01-28,01:00,26.0,Passing clouds.,6,0.0,74.0,1010.0,5,2483.37,101.92,54.84,1.16,125.89,130.48,164.70,22.04
1513,2021-01-28 02:00:00,2021-01-28,02:00,25.0,Passing clouds.,6,140.0,79.0,1010.0,5,2723.69,102.82,58.95,2.82,127.79,135.38,172.46,22.55
1514,2021-01-28 03:00:00,2021-01-28,03:00,24.0,Passing clouds.,6,80.0,83.0,1010.0,5,2536.77,77.78,64.43,3.71,108.72,123.73,160.45,21.28
1515,2021-01-28 04:00:00,2021-01-28,04:00,24.0,Passing clouds.,6,70.0,89.0,1010.0,5,1522.06,16.32,59.63,17.88,51.50,82.38,105.37,9.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37458,2025-03-27 19:00:00,2025-03-27,19:00,29.0,Clear.,17,150.0,58.0,1004.0,2,741.00,5.53,32.90,1.08,30.99,21.93,34.68,6.02
37459,2025-03-27 20:00:00,2025-03-27,20:00,28.0,Clear.,17,150.0,70.0,1005.0,3,801.09,9.16,31.19,0.31,30.99,25.97,38.24,6.59
37460,2025-03-27 21:00:00,2025-03-27,21:00,27.0,Passing clouds.,15,160.0,74.0,1006.0,3,881.20,13.30,30.16,0.08,31.95,30.32,42.52,7.03
37461,2025-03-27 22:00:00,2025-03-27,22:00,27.0,Passing clouds.,11,140.0,79.0,1006.0,3,961.30,19.67,30.50,0.01,36.72,35.01,47.68,7.22


In [494]:
merge_data[merge_data['date'] == '2021-01-05']

,datetime,date,time,temp,weather,wind,wind_direction,humidity,pressure,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
983,2021-01-05 00:00:00,2021-01-05,00:00,26.0,Passing clouds.,9,10.0,79.0,1010.0,4,921.25,25.26,34.62,0.08,56.27,32.23,39.55,7.92
984,2021-01-05 01:00:00,2021-01-05,01:00,25.0,Passing clouds.,7,30.0,83.0,1010.0,4,1321.79,39.34,38.04,1.86,72.48,42.81,53.69,10.77
985,2021-01-05 02:00:00,2021-01-05,02:00,25.0,Passing clouds.,4,0.0,83.0,1009.0,5,1615.52,54.09,41.81,6.08,92.51,52.91,68.12,14.57
986,2021-01-05 03:00:00,2021-01-05,03:00,25.0,Warm.,7,10.0,83.0,1009.0,5,1789.09,59.01,56.21,15.20,116.35,64.10,83.86,17.99
987,2021-01-05 04:00:00,2021-01-05,04:00,NaN,NaN,NaN,NaN,NaN,NaN,4,1054.76,17.21,50.04,60.08,73.43,40.75,51.89,9.75
988,2021-01-05 05:00:00,2021-01-05,05:00,24.0,Passing clouds.,6,0.0,83.0,1010.0,4,734.33,5.48,33.59,101.57,52.45,30.51,37.82,6.40
989,2021-01-05 06:00:00,2021-01-05,06:00,24.0,Passing clouds.,6,30.0,83.0,1010.0,3,560.76,2.54,21.42,111.58,37.19,21.51,26.54,4.62
990,2021-01-05 07:00:00,2021-01-05,07:00,24.0,Passing clouds.,4,0.0,89.0,1011.0,2,500.68,2.26,18.51,100.14,30.52,15.34,18.87,3.99
991,2021-01-05 08:00:00,2021-01-05,08:00,26.0,Passing clouds.,7,60.0,79.0,1012.0,2,520.71,2.60,21.94,92.98,33.85,14.76,18.07,4.31
992,2021-01-05 09:00:00,2021-01-05,09:00,28.0,Passing clouds.,7,20.0,70.0,1012.0,2,554.09,2.52,25.71,83.69,36.24,14.45,18.05,4.69


In [ ]:
#save to csv


In [500]:
merge_data.to_csv('final_data.csv', index=False)